<a href="https://colab.research.google.com/github/Dylan-Morrissey/Fat-Prediction-Neural-Network/blob/master/Bodyfat_Predicton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import random
import glob
from keras.utils import np_utils
from tensorflow import keras
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from sklearn.model_selection import train_test_split
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import concatenate
from keras.layers import Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model

print(tf.__version__)

2.1.0


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

cols = ["Gender", "Age", "Weight", "Height", "Range","Bodyfat"]
inputfile = "/content/gdrive/My Drive/Bodyfat Dataset/UsersInfo.txt"
df = pd.read_csv(inputfile, sep=" ", header=None, names=cols)

#df['Bodyfat'] = df['Bodyfat'].astype(int)
df.head()
df = df.dropna()
df.shape

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df.isnull().any()

(322, 6)

In [0]:
df.dtypes

Gender     False
Age        False
Weight     False
Height     False
Range      False
Bodyfat    False
dtype: bool

In [0]:
df = pd.DataFrame(df)
df = df.drop(columns=["Gender", "Range"])
df['Height'] = df['Height'].astype(float)
df.head()

21.0    56
19.0    51
11.0    31
14.0    29
16.0    27
13.0    25
17.0    24
25.0    22
18.0    16
15.0     9
9.0      8
10.0     7
7.0      5
8.0      4
20.0     2
12.0     2
45.0     1
30.0     1
23.0     1
24.0     1
Name: Bodyfat, dtype: int64

In [0]:
import pandas_profiling
pandas_profiling.ProfileReport(df)


/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,7
Number of observations,322
Total Missing (%),0.0%
Total size in memory,17.7 KiB
Average record size in memory,56.4 B
Numeric,4
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [0]:
images = []
inputPath = "/content/gdrive/My Drive/Bodyfat Dataset/"
from PIL import Image
import glob
import cv2
for file in glob.glob(inputPath+"*.jpg"):
  image = cv2.imread(file)
  image = cv2.resize(image,(200,200))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  images.append(image)
  train_images = np.array(images)

In [0]:
train_images = train_images.reshape(train_images.shape[0],200,200,1).astype("float32")
train_images = train_images / 255

In [0]:
maxAge = df['Age'].max()
df['Age'] = df['Age'] / maxAge
maxWeight = df['Weight'].max()
df['Weight'] = df['Weight'] / maxWeight
maxHeight = df['Height'].max()
df['Height'] = df['Height'] / maxHeight

df.head()

Train on 322 samples
Epoch 1/15
322/322 [==============================] - 3s 8ms/sample - loss: 1.0814 - accuracy: 0.0000e+00
Epoch 2/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.3529 - accuracy: 0.0000e+00
Epoch 3/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.1115 - accuracy: 0.0000e+00
Epoch 4/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0267 - accuracy: 0.0000e+00
Epoch 5/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0186 - accuracy: 0.0000e+00
Epoch 6/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0191 - accuracy: 0.0000e+00
Epoch 7/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0145 - accuracy: 0.0000e+00
Epoch 8/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0144 - accuracy: 0.0000e+00
Epoch 9/15
322/322 [==============================] - 2s 6ms/sample - loss: 0.0168 - accuracy: 0.0000e+00
Epoch 10/15
322/322 [====

In [0]:
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))

	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))

	# return our model
	return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input
	inputs = Input(shape=inputShape)

	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs

		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)

	# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)

	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)

	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)

	# construct the CNN
	model = Model(inputs, x)

	# return the CNN
	return model

In [0]:
split = train_test_split(df,train_images, test_size=0.25, random_state=42)
(x_train, x_test, train_Images, test_Images) = split
maxBF = x_train['Bodyfat'].max()
y_train = x_train['Bodyfat'] / maxBF
y_test = x_test['Bodyfat'] / maxBF

In [0]:
mlp = create_mlp(df.shape[1], regress=False)

In [0]:
cnn = create_cnn(200,200,1,regress=False)

In [0]:
combinedInput = concatenate([mlp.output, cnn.output])

In [0]:
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [0]:
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [0]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [0]:
print("[INFO] training model...")
model.fit(
	[x_train, train_Images], y_train,
	validation_data=([x_test, test_Images], y_test),
	epochs=200, batch_size=8)

In [0]:
preds = model.predict([testAttrX, testImagesX])

In [0]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

In [0]:
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [0]:
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))